In [2]:
from neo4j import GraphDatabase

URI = 'neo4j://localhost'
AUTH = ("neo4j", "password")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

db_driver = GraphDatabase.driver(URI, auth=AUTH)

In [3]:
import os
config_paths = '/home/narain/data/config_jsons/JsonData'
config_fs = list(filter(lambda x: x.endswith('.json'), 
        map(lambda x: os.path.join(config_paths, x), os.listdir(config_paths))))
len(os.listdir(config_paths))

3765

In [4]:
import json, re, fastprogress
from pathlib import Path

In [6]:
with open('acc_dict.json') as f:
    acc_dict = json.load(f)

### Delete all notes and connections

In [16]:
with db_driver.session() as db_session:
    db_session.run('match (a) -[r] -> () delete a, r')
    db_session.run('match (a) delete a')

### Create Nodes boards panel board vlmccb

In [17]:
with db_driver.session() as db_session:
    for k in acc_dict.keys():
        board = k.split(":")[1]
        db_session.run(f'CREATE (n:TYZ {{name: "{board}"}})')

### Create Component Nodes

In [18]:
components = []
for k,v in acc_dict.items():
    components.extend(list(v.keys()))

with db_driver.session() as db_session:
    for k in set(components):
        db_session.run(f'CREATE ({k}:COMP {{name: "{k}"}})')

### Create CSTIC values nodes

In [19]:
with db_driver.session() as db_session:
    for k,v in  acc_dict.items():
        for k1, v1 in v.items():
            for v2 in v1:
                db_session.run(f'CREATE ({k1}:VAL {{value: "{v2}", category: "{k1}"}})')

In [20]:
Path(config_fs[0]).name

'RAMOCD1_0X37877E1C25116C19.json'

### Create Induvidual graphs

In [21]:
with db_driver.session() as db_session:
    for f in config_fs:
        f_name = Path(f).name[:-5]
        db_session.run(f'CREATE (n:G {{name: "{f_name}"}})')

In [22]:
# with db_driver.session() as db_session:
#     o = db_session.run(f'MATCH (n:TYZ) \nRETURN *')
#     print(o.values())

In [24]:
def read_config(file):
    with open(file) as f:
        o = json.load(f)
    return o

## Create Connnections

In [65]:
with db_driver.session() as db_session:
    for file in fastprogress.progress_bar(config_fs, total=len(config_fs)):
        f_name = Path(file).name[:-5
        content = read_config(file)
        node_dict = dict()
        for ele in o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE']:
            ele_number = 'parent' if ele['parent'] == '-1' else ele['parent']
            node_dict[ele['number']] = ele['name']
            name = node_dict[ele['parent']] if ele['parent'] in node_dict else ele['parent']
            parent_node = f'n:G {{name: "{f_name[:-5]}"}}' if ele['parent'] == '-1' else f'n:TYZ {{name: "{node_dict[ele["parent"]].split(":")[-1]}"}}' 
            current_node = f'm:TYZ {{name: "{ele["name"].split(":")[-1]}"}}'
            db_session.run(f'MATCH ({parent_node}), ({current_node}) CREATE (n)-[:CHILD_{f_name}_{ele_number}]->(m)')
            for item in ele['CSTIC']:
                item_name, item_value = itesm['name'], item['value']
                comp_node = f'c:COMP {{name: "{item["name"]}"}}'
                db_session.run(f"MATCH ({current_node}), ({comp_node}) CREATE (m)-[:COMP_{f_name}_{ele['number']}]->(c)")
                value_node = f'v:VAL {{name: "{item_value}", category: "{item_name}"}}'
                db_session.run(f"MATCH ({comp_node}), ({value_node}) CREATE (c)-[:VALUE_{f_name}_{ele['number']}]->(v)")

KeyboardInterrupt: 

In [64]:
'parent' if ele['parent'] == '-1'

'-1'

In [63]:
ele_number

'-1'

In [54]:
for item in ele['CSTIC']:
    item_name, item_value = item['name'], item['value']
    comp_node = f'c:COMP {{name: "{item["name"]}"}}'
    print(f"MATCH {current_node}, {comp_node} CREATE (m)-[:COMP_{f_name}_{ele['number']}]->(c)")
    value_node = f'v:VAL {{name: "{item_value}", category: "{item_name}"}}'
    print(f"MATCH {comp_node}, {value_node} CREATE (c)-[:VALUE_{f_name}_{ele['number']}]->(v)")

MATCH m:TYZ {name: PB_VL}, c:COMP {name: "A7J_CATALOG_NUMBER"} CREATE (m)-[:COMP_RAMOCD1_0X37877E1C25116C19_22]->(c)
MATCH c:COMP {name: "A7J_CATALOG_NUMBER"}, v:VAL {name: "LGB3B100B", category: "A7J_CATALOG_NUMBER"} CREATE (c)-[:VALUE_RAMOCD1_0X37877E1C25116C19_22]->(v)
MATCH m:TYZ {name: PB_VL}, c:COMP {name: "A7J04_ELECTRONIC_TRIP_VERSION"} CREATE (m)-[:COMP_RAMOCD1_0X37877E1C25116C19_22]->(c)
MATCH c:COMP {name: "A7J04_ELECTRONIC_TRIP_VERSION"}, v:VAL {name: "", category: "A7J04_ELECTRONIC_TRIP_VERSION"} CREATE (c)-[:VALUE_RAMOCD1_0X37877E1C25116C19_22]->(v)
MATCH m:TYZ {name: PB_VL}, c:COMP {name: "A7J04_MARINE_50C_CALIBRATION"} CREATE (m)-[:COMP_RAMOCD1_0X37877E1C25116C19_22]->(c)
MATCH c:COMP {name: "A7J04_MARINE_50C_CALIBRATION"}, v:VAL {name: "NO", category: "A7J04_MARINE_50C_CALIBRATION"} CREATE (c)-[:VALUE_RAMOCD1_0X37877E1C25116C19_22]->(v)
MATCH m:TYZ {name: PB_VL}, c:COMP {name: "A7J04_LONG_TIME"} CREATE (m)-[:COMP_RAMOCD1_0X37877E1C25116C19_22]->(c)
MATCH c:COMP {name: 